pandas：数据分析工具，与张量兼容，数据类型是DataFrame key-value的list集合
处理缺失值方法：插值法和删除法
数值存在NaN也可以转化为tensor格式，一般全是数值类型才进行转化

In [1]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True) # 创建多层目录，exist_ok=True 参数表示如果目录已经存在，则不会引发任何错误，反之会出现错误
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n') # 列名
    f.write('NA,Pave,127500\n') # 每行是个数据样本，NA是表示NaN
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [2]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


缺失值处理
数值缺失：未缺失的均值替换  inputs.fillna()
类别值或离散值缺失：NaN视为一个类别  pd.get_dummies(inputs, dummy_na=True)

In [3]:
# iloc：index location 对数据进行位置索引
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
# fillna(a) 将数组中的空值填充为指定的值 a
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


In [4]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


inputs和outputs转为张量格式
获取inputs和outputs值
inputs.values
outputs.values

In [5]:
import torch
x, y = torch.tensor(inputs.values), torch.tensor(outputs.values) # float64 默认 float32 常用
x, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

删除缺失值最多的列

In [6]:
inputs_del = data.iloc[:, 0:2]
inputs_na = inputs_del.isna() # 获取data的缺失值布尔矩阵
inputs_na

,NumRooms,Alley
0,True,False
1,False,True
2,False,True
3,True,True


In [7]:
index_sum = inputs_na.sum() # 求出每列缺失值的数量
index_sum

NumRooms    2
Alley       3
dtype: int64

In [8]:
index = index_sum.idxmax() # 获取缺失值最多的列key
index

'Alley'

In [9]:
inputs_drop = inputs_del.drop(index, axis=1)
print(inputs_drop)

   NumRooms
0       NaN
1       2.0
2       4.0
3       NaN


In [10]:
inputs = inputs_drop.fillna(inputs_drop.mean())
inputs_tensor = torch.tensor(inputs.values)
inputs_tensor

tensor([[3.],
        [2.],
        [4.],
        [3.]], dtype=torch.float64)